In [1]:
import nltk
nltk.download('stopwords')
#!pip install imblearn

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
%matplotlib inline

import re
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix,precision_recall_fscore_support
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from imblearn.pipeline import Pipeline
import seaborn as sns
from imblearn.over_sampling import SMOTE

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
import os
os.chdir('/content/drive/My Drive/NLP_Project')

In [5]:
df_train = pd.read_csv('train.tsv',sep='\t')
df_test = pd.read_csv('dev.tsv',sep='\t')

In [7]:
df_train.head()

,Unnamed: 0,ids,body,Uncertainity of Post_Diagnosis,Results and Side-Effects Observed,Medical Assistance,Diet and Maintenance,Information Source,Concepts
0,0,LIPITOR.449.txt,Extreme tiredness and flatulence. Not sure whe...,0,1,0,0,0,flatulence|exhaustion|tired|Not sure|Extreme|t...
1,1,LIPITOR.188.txt,1/7/05-continued. not all of it posted before....,0,1,0,1,0,package insert|depression|Lipitor|cholesterol|...
2,2,LIPITOR.541.txt,So sad to see so many with problems like mine!...,1,1,1,0,0,muscle pain|joint pain|depression|Lipitor|Lipi...
3,3,LIPITOR.810.txt,Within 1 month time developed severe depressio...,0,1,0,0,0,severe depression|headaches|Lipitor|statins|Li...
4,4,LIPITOR.393.txt,I have been on lipitor for 10 years for heart ...,0,1,0,0,0,leg weakness|changed|experience|Potassium|cram...


In [8]:
df_test.head()

,Unnamed: 0,ids,body,Uncertainity of Post_Diagnosis,Results and Side-Effects Observed,Medical Assistance,Diet and Maintenance,Information Source,Concepts
0,0,LIPITOR.595.txt,"Swelling left arm, very severe itching, intole...",0,1,0,0,0,Swelling|itching|left arm|hand|dreams|bruise|v...
1,1,ARTHROTEC.101.txt,"1st pill taken with food, a few hours after i ...",0,1,0,0,0,side effects|depression|cramping|upset|experie...
2,2,LIPITOR.367.txt,episode of intense dizziness lasting nearly an...,0,1,0,0,0,dizziness|lassitude|chills|shivers|problem|wor...
3,3,LIPITOR.74.txt,After taking Crestor and having muscle pain an...,0,1,0,0,1,liver problems|decreased|itchy|control|crawly|...
4,4,LIPITOR.389.txt,"75 yo mother-in-law has memory loss, hair loss...",0,1,0,1,0,lack of appetite|hair loss|stroke|sciatica|los...


In [9]:
df_train = df_train.drop(['Unnamed: 0','Concepts','ids'],axis = 1)

In [10]:
df_test = df_test.drop(['Unnamed: 0','Concepts','ids'],axis = 1)

In [11]:
df_train.head()

,body,Uncertainity of Post_Diagnosis,Results and Side-Effects Observed,Medical Assistance,Diet and Maintenance,Information Source
0,Extreme tiredness and flatulence. Not sure whe...,0,1,0,0,0
1,1/7/05-continued. not all of it posted before....,0,1,0,1,0
2,So sad to see so many with problems like mine!...,1,1,1,0,0
3,Within 1 month time developed severe depressio...,0,1,0,0,0
4,I have been on lipitor for 10 years for heart ...,0,1,0,0,0


In [12]:
df_test.head()

,body,Uncertainity of Post_Diagnosis,Results and Side-Effects Observed,Medical Assistance,Diet and Maintenance,Information Source
0,"Swelling left arm, very severe itching, intole...",0,1,0,0,0
1,"1st pill taken with food, a few hours after i ...",0,1,0,0,0
2,episode of intense dizziness lasting nearly an...,0,1,0,0,0
3,After taking Crestor and having muscle pain an...,0,1,0,0,1
4,"75 yo mother-in-law has memory loss, hair loss...",0,1,0,1,0


In [13]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

In [14]:
categories = ['Uncertainity of Post_Diagnosis', 'Results and Side-Effects Observed', 'Medical Assistance', 'Diet and Maintenance', 'Information Source']

In [15]:
df_train['body'] = df_train['body'].map(lambda com : clean_text(com))
df_test['body'] = df_test['body'].map(lambda com : clean_text(com))
df_train

,body,Uncertainity of Post_Diagnosis,Results and Side-Effects Observed,Medical Assistance,Diet and Maintenance,Information Source
0,extreme tiredness and flatulence not sure whet...,0,1,0,0,0
1,1 7 05 continued not all of it posted before h...,0,1,0,1,0
2,so sad to see so many with problems like mine ...,1,1,1,0,0
3,within 1 month time developed severe depressio...,0,1,0,0,0
4,i have been on lipitor for 10 years for heart ...,0,1,0,0,0
...,...,...,...,...,...,...
937,i had diarrhea for a whole week bad uterine cr...,0,1,0,0,0
938,i have never been so miserable in my life star...,0,1,0,1,0
939,severe muscle and joint pain in left leg my ca...,0,1,0,0,0
940,gradual but significant increase in sleep requ...,0,1,0,0,0


In [16]:
X_train = df_train.body
X_test = df_test.body
print(X_train.shape)
print(X_test.shape)

(942,)
(300,)


In [17]:
smt = SMOTE(random_state=42)

In [19]:
NB_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('smt',smt),
                ('clf', OneVsRestClassifier(MultinomialNB(
                    fit_prior=True, class_prior=None))),
            ])
print('... Processing {}'.format('Uncertainity of Post_Diagnosis'))
    # train the model using X_dtm & y
NB_pipeline.fit(X_train, df_train['Uncertainity of Post_Diagnosis'])
# compute the testing accuracy
prediction_NB = NB_pipeline.predict(X_test)
print('Test accuracy is {}'.format(accuracy_score(df_test['Uncertainity of Post_Diagnosis'], prediction_NB)))
print(classification_report(df_test['Uncertainity of Post_Diagnosis'],prediction_NB))

... Processing Uncertainity of Post_Diagnosis


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Test accuracy is 0.54
              precision    recall  f1-score   support

           0       0.94      0.49      0.65       257
           1       0.21      0.81      0.34        43

    accuracy                           0.54       300
   macro avg       0.58      0.65      0.49       300
weighted avg       0.84      0.54      0.60       300



In [20]:
LogReg_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('smt',smt),
                ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=1)),
            ])
print('... Processing {}'.format('Uncertainity of Post_Diagnosis'))
# train the model using X_dtm & y
LogReg_pipeline.fit(X_train, df_train['Uncertainity of Post_Diagnosis'])
# compute the testing accuracy
prediction_LogReg = LogReg_pipeline.predict(X_test)
print('Test accuracy is {}'.format(accuracy_score(df_test['Uncertainity of Post_Diagnosis'], prediction_LogReg)))
print(classification_report(df_test['Uncertainity of Post_Diagnosis'],prediction_LogReg))

... Processing Uncertainity of Post_Diagnosis


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Test accuracy is 0.8033333333333333
              precision    recall  f1-score   support

           0       0.89      0.88      0.88       257
           1       0.33      0.37      0.35        43

    accuracy                           0.80       300
   macro avg       0.61      0.62      0.62       300
weighted avg       0.81      0.80      0.81       300



In [21]:
table=[]

In [22]:
a11=0
a12=0
for i in range(300):
  if(df_test['Uncertainity of Post_Diagnosis'][i]==prediction_NB[i] and df_test['Uncertainity of Post_Diagnosis'][i]==prediction_LogReg[i]):
    a11=a11+1
  if(df_test['Uncertainity of Post_Diagnosis'][i]==prediction_NB[i] and df_test['Uncertainity of Post_Diagnosis'][i]!=prediction_LogReg[i]):
    a12=a12+1
table.append([a11,a12])

In [23]:
a21=0
a22=0
for i in range(300):
  if(df_test['Uncertainity of Post_Diagnosis'][i]!=prediction_NB[i] and df_test['Uncertainity of Post_Diagnosis'][i]==prediction_LogReg[i]):
    a21=a21+1
  if(df_test['Uncertainity of Post_Diagnosis'][i]!=prediction_NB[i] and df_test['Uncertainity of Post_Diagnosis'][i]!=prediction_LogReg[i]):
    a22=a22+1
table.append([a21,a22])

In [24]:
table

[[143, 19], [98, 40]]

In [29]:
from statsmodels.stats.contingency_tables import mcnemar
result = mcnemar(table, exact=False, correction=True)
print('statistic=%.3f, p-value=%.3f' % (result.statistic, result.pvalue))
alpha = 0.05
if result.pvalue > alpha:
	print('Same proportions of errors (fail to reject H0)')
else:
	print('Different proportions of errors (reject H0)')

statistic=52.000, p-value=0.000
Different proportions of errors (reject H0)
